In [5]:
import pandas as pd
import bs4
import selenium
import requests
import re
from selenium.common.exceptions import NoSuchElementException

# Building Recoomendation System

In [6]:
from selenium import webdriver

In [7]:
browser = webdriver.Chrome('chromedriver.exe')
browser.get('https://www.zomato.com/bangalore/great-food-no-bull')

In [8]:
restaurants =browser.find_elements_by_css_selector('div.col-s-8.col-l-1by3')
print(len(restaurants))

132


In [9]:
def get_info(review, selector , attribute):
    try:    
        tag = review.find_element_by_css_selector(selector)
    
        tag = tag.get_attribute(attribute)
    except:
        tag=None
    return tag
df_restaurants=pd.DataFrame(columns={'rest_name','rest_link'})
for restaurant in restaurants:  
    rest_name = get_info(restaurant,'div.res_title.zblack.bold.nowrap','innerHTML')
    if rest_name != None:
        rest_name = rest_name.strip()
    rest_link = get_info(restaurant,'a.relative.top-res-box-bg.pl10.ptop0','href')
    curr_restaurant={
                 'rest_name':rest_name,
                 'rest_link':rest_link
             }
    df_restaurants= df_restaurants.append(curr_restaurant,ignore_index=True)
df_restaurants.head()

,rest_name,rest_link
0,Brewklyn Microbrewery,https://www.zomato.com/bangalore/brewklyn-micr...
1,Output Bengaluru,https://www.zomato.com/bangalore/output-bengal...
2,Punjab Bistro,https://www.zomato.com/bangalore/punjab-bistro...
3,The Old Fashioned Bar,https://www.zomato.com/bangalore/the-old-fashi...
4,1131 Bar + Kitchen,https://www.zomato.com/bangalore/1131-bar-kitc...


In [55]:
df_reviews=pd.DataFrame(columns={'rating','user_name','user_id','rest_name'})

In [56]:
for link in df_restaurants['rest_link']:
    browser.get(link )
    #review_sel ='a.item.respageMenu-item.restaurant-tab-item-jumbo-track.active'
    #review_button = browser.find_element_by_css_selector(review_sel)
    #review_button.click()
    while True:
        more_sel = 'div.load-more'
        try:
            load_more_button = browser.find_element_by_css_selector(more_sel)
            load_more_button.click()

            reviews = browser.find_elements_by_css_selector('div.ui.segment.clearfix.brtop')
        except Exception:
            break
    def get_info(review, selector , attribute):
        try:    
            tag = review.find_element_by_css_selector(selector)

            tag = tag.get_attribute(attribute)
        except:
            tag=None
        return tag
        
    for review in reviews:  
            rating=get_info(review,'div.ttupper','aria-label')
            #y=re.findall('[0-9.]+',rating)
            #rating = float(y[0])
            name_selec = 'a[data-entity_id]'
            user_id =get_info(review,name_selec,'data-entity_id')  
            user_name = get_info(review,name_selec,'innerHTML') 
            if user_name !=None:
                user_name = user_name.strip()
            rest_name = get_info(browser,'a.ui.large.header','title') 
            curr_review={
                         'user_name':user_name,
                         'rating':rating,
                         'user_id':user_id,
                         'rest_name':rest_name
                     }
            df_reviews= df_reviews.append(curr_review,ignore_index=True)
df_reviews.head()  

,rest_name,rating,user_id,user_name
0,Brewklyn Microbrewery Bangalore,Rated 5.0,1826583,Reedhika Puliani
1,Brewklyn Microbrewery Bangalore,Rated 5.0,31021666,Nikhil Singla
2,Brewklyn Microbrewery Bangalore,Rated 5.0,3048280,Alexandrina Namrata
3,Brewklyn Microbrewery Bangalore,Rated 4.0,36088032,Abhishek Sharma
4,Brewklyn Microbrewery Bangalore,Rated 4.0,407847,Nitin Hajela (TheFoodieTales.com)


In [78]:
df_reviews.shape[0]

7974

In [63]:
df_reviews.to_csv('zomato.csv',index=False)